# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,176,177,178,179,180,181,182,183,184,185
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
n_features = (X_train.shape[1],)

print(n_features)

(222,)


In [6]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=n_features))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150,
                   validation_data=(X_val, y_val))

Metal device set to: Apple M1 Pro
Epoch 1/150


2023-04-14 18:33:37.652863: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-14 18:33:37.652988: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-14 18:33:37.784944: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


11/33 [=========>....................] - ETA: 0s - loss: nan - mse: nan                          

2023-04-14 18:33:37.882642: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 11ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
21/33 [==================>...........] - ETA: 0s - loss: nan - mse: nan

2023-04-14 18:33:38.276389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 [==============================] - 0s 6ms

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 78/150
33/33 [==============================] -

33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 146/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [9]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150,
                           validation_data=(X_val, y_val))

Epoch 1/150
24/33 [====================>.........] - ETA: 0s - loss: nan - mse: nan                         

2023-04-14 18:34:06.738799: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 9ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
25/33 [=====================>........] - ETA: 0s - loss: nan - mse: nan

2023-04-14 18:34:07.021691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 [==============================] - 0s 5ms

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 78/150
33/33 [==============================] -

33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 146/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150,
                     validation_data=(X_val, y_val))

Epoch 1/150
24/33 [====================>.........] - ETA: 0s - loss: 0.5549 - mse: 0.5549

2023-04-14 18:34:34.584020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 9ms/step - loss: 0.4699 - mse: 0.4699 - val_loss: 36093075456.0000 - val_mse: 36093075456.0000
Epoch 2/150
25/33 [=====================>........] - ETA: 0s - loss: 0.2978 - mse: 0.2978

2023-04-14 18:34:34.868520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.2649 - mse: 0.2649 - val_loss: 36093042688.0000 - val_mse: 36093042688.0000
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1990 - mse: 0.1990 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1719 - mse: 0.1719 - val_loss: 36093054976.0000 - val_mse: 36093054976.0000
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1535 - mse: 0.1535 - val_loss: 36093050880.0000 - val_mse: 36093050880.0000
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1312 - mse: 0.1312 - val_loss: 36093046784.0000 - val_mse: 36093046784.0000
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1308 - mse: 0.1308 - val_loss: 36093050880.0000 - val_mse: 36093050880.0000
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1182 - mse: 0.1182 - val_loss: 

33/33 [==============================] - 0s 5ms/step - loss: 0.0268 - mse: 0.0268 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 57/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0262 - mse: 0.0262 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 58/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 59/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 60/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 61/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 62/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0239 - mse: 0.0239 - val_

Epoch 110/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 36093067264.0000 - val_mse: 36093067264.0000
Epoch 111/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 112/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 113/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 36093071360.0000 - val_mse: 36093071360.0000
Epoch 114/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 36093067264.0000 - val_mse: 36093067264.0000
Epoch 115/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 116/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0117 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0077 - mse: 0.0077


2023-04-14 18:35:01.752247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[0.007683976087719202, 0.007683976087719202]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

2023-04-14 18:35:01.949149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 3ms/step


28123.691471632756

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150


2023-04-14 18:35:02.098586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 9ms/step - loss: 0.5188 - mse: 0.5188 - val_loss: 0.3186 - val_mse: 0.3186
Epoch 2/150
22/33 [===================>..........] - ETA: 0s - loss: 0.2668 - mse: 0.2668

2023-04-14 18:35:02.379081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.2756 - mse: 0.2756 - val_loss: 0.1981 - val_mse: 0.1981
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.2248 - mse: 0.2248 - val_loss: 0.1703 - val_mse: 0.1703
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1884 - mse: 0.1884 - val_loss: 0.1575 - val_mse: 0.1575
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1721 - mse: 0.1721 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1590 - mse: 0.1590 - val_loss: 0.1417 - val_mse: 0.1417
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1484 - mse: 0.1484 - val_loss: 0.1515 - val_mse: 0.1515
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1400 - mse: 0.1400 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1274 - mse: 0.1274 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0253 - mse: 0.0253 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1207 - val_mse: 0.1207
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1188 - val_mse: 0.1188
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.1198 - val_mse: 0.1198
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0223 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1172 - val_mse: 0.1172
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1164 - val_mse: 0.1164
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1177 - val_mse: 0.1177
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1171 - val_mse: 0.1171
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0090 - mse: 0.0090


[0.008950877003371716, 0.008950877003371716]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1166 - mse: 0.1166


[0.11658976972103119, 0.11658976972103119]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
24/33 [====================>.........] - ETA: 0s - loss: 0.5655 - mse: 0.5655

2023-04-14 18:35:29.370135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 9ms/step - loss: 0.4588 - mse: 0.4588 - val_loss: 0.1934 - val_mse: 0.1934
Epoch 2/150
25/33 [=====================>........] - ETA: 0s - loss: 0.2342 - mse: 0.2342

2023-04-14 18:35:29.650954: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.2253 - mse: 0.2253 - val_loss: 0.1639 - val_mse: 0.1639
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1780 - mse: 0.1780 - val_loss: 0.1459 - val_mse: 0.1459
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1511 - mse: 0.1511 - val_loss: 0.1429 - val_mse: 0.1429
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1360 - mse: 0.1360 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1260 - mse: 0.1260 - val_loss: 0.1341 - val_mse: 0.1341
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1134 - mse: 0.1134 - val_loss: 0.1316 - val_mse: 0.1316
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1043 - mse: 0.1043 - val_loss: 0.1248 - val_mse: 0.1248
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0978 - mse: 0.0978 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1411 - val_mse: 0.1411
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0204 - mse: 0.0204 - val_loss: 0.1429 - val_mse: 0.1429
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.1426 - val_mse: 0.1426
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0195 - mse: 0.0195 - val_loss: 0.1440 - val_mse: 0.1440
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0198 - mse: 0.0198 - val_loss: 0.1443 - val_mse: 0.1443
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0193 - mse: 0.0193 - val_loss: 0.1445 - val_mse: 0.1445
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0188 - mse: 0.0188 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0184 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1525 - val_mse: 0.1525
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1523 - val_mse: 0.1523
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1505 - val_mse: 0.1505
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 130/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1518 - val_mse: 0.1518
Epoch 131/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1513 - val_mse: 0.1513
Epoch 132/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1513 - val_mse: 0.1513
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0080 - mse: 0.0080


[0.00799749419093132, 0.00799749419093132]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1555 - mse: 0.1555


[0.15551191568374634, 0.15551191568374634]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
 1/33 [..............................] - ETA: 9s - loss: 0.9326 - mse: 0.9326

2023-04-14 18:35:56.957068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 14ms/step - loss: 0.3377 - mse: 0.3377 - val_loss: 0.1505 - val_mse: 0.1505
Epoch 2/150
 9/33 [=======>......................] - ETA: 0s - loss: 0.1055 - mse: 0.1055

2023-04-14 18:35:57.500517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 7ms/step - loss: 0.1919 - mse: 0.1919 - val_loss: 0.1563 - val_mse: 0.1563
Epoch 3/150
33/33 [==============================] - 0s 8ms/step - loss: 0.1279 - mse: 0.1279 - val_loss: 0.1524 - val_mse: 0.1524
Epoch 4/150
33/33 [==============================] - 0s 8ms/step - loss: 0.1018 - mse: 0.1018 - val_loss: 0.2272 - val_mse: 0.2272
Epoch 5/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0747 - mse: 0.0747 - val_loss: 0.1330 - val_mse: 0.1330
Epoch 6/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0726 - mse: 0.0726 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 7/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0435 - mse: 0.0435 - val_loss: 0.1245 - val_mse: 0.1245
Epoch 8/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0490 - mse: 0.0490 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 9/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0353 - mse: 0.0353 -

33/33 [==============================] - 0s 7ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.0970 - val_mse: 0.0970
Epoch 65/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0838 - val_mse: 0.0838
Epoch 66/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.0853 - val_mse: 0.0853
Epoch 67/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0869 - val_mse: 0.0869
Epoch 68/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0921 - val_mse: 0.0921
Epoch 69/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.0799 - val_mse: 0.0799
Epoch 70/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.0842 - val_mse: 0.0842
Epoch 71/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0056 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0940 - val_mse: 0.0940
Epoch 127/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0062 - mse: 0.0062 - val_loss: 0.0813 - val_mse: 0.0813
Epoch 128/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0772 - val_mse: 0.0772
Epoch 129/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0772 - val_mse: 0.0772
Epoch 130/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0781 - val_mse: 0.0781
Epoch 131/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.0813 - val_mse: 0.0813
Epoch 132/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0811 - val_mse: 0.0811
Epoch 133/150
33/33 [==============================] - 0s 8ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0022 - mse: 0.0022


[0.002239547437056899, 0.002239547437056899]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [22]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.0863 - mse: 0.0863


[0.0863339826464653, 0.0863339826464653]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
 9/33 [=======>......................] - ETA: 0s - loss: 0.6130 - mse: 0.6130

2023-04-14 18:36:34.149293: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 10ms/step - loss: 0.4503 - mse: 0.4503 - val_loss: 0.1665 - val_mse: 0.1665
Epoch 2/150
22/33 [===================>..........] - ETA: 0s - loss: 0.1821 - mse: 0.1821

2023-04-14 18:36:34.495092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.1841 - mse: 0.1841 - val_loss: 0.1547 - val_mse: 0.1547
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1180 - mse: 0.1180 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0901 - mse: 0.0901 - val_loss: 0.1400 - val_mse: 0.1400
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0703 - mse: 0.0703 - val_loss: 0.1088 - val_mse: 0.1088
Epoch 6/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0529 - mse: 0.0529 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0416 - mse: 0.0416 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0347 - mse: 0.0347 - val_loss: 0.1276 - val_mse: 0.1276
Epoch 9/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0291 - mse: 0.0291 -

33/33 [==============================] - 0s 6ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 65/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1237 - val_mse: 0.1237
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 67/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 68/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 69/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0034 - mse: 0.0034 - val_loss: 0.1067 - val_mse: 0.1067
Epoch 70/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.1223 - val_mse: 0.1223
Epoch 71/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0054 - mse: 0

33/33 [==============================] - 0s 6ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 126/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1111 - val_mse: 0.1111
Epoch 127/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0015 - mse: 0.0015 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 128/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 130/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.1050 - val_mse: 0.1050
Epoch 131/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 132/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0021 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 8.5762e-04 - mse: 8.5762e-04


[0.0008576181717216969, 0.0008576181717216969]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1052 - mse: 0.1052


[0.105233334004879, 0.105233334004879]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [26]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 7ms/step - loss: 0.2161 - mse: 0.2161


[0.21607691049575806, 0.21607691049575806]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [27]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

5/5 [==============================] - 0s 4ms/step


2023-04-14 18:37:05.993815: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


36528.82298654173

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.